# Algoritmo GRASP


## Leyendo datos de TSPData

Primero vamos a coger una distribución de ciudades reales, en particular de Argentina. 

Mostremos el texto (para enseñar el formato)

In [7]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [8]:
import pandas as pd

In [9]:
# Tamaño de ciudades
num_ciudades=1000


In [10]:
from StringIO import StringIO

df = pd.read_table('ar9152.tsp', sep=' ', skiprows=7, skipfooter=1, names=['x','y'], engine='python')

df = df[:-1]
print df.tail()

               x           y
9147  34383.0556  58848.8889
9148  34411.1111  58798.3333
9149  34813.3333  58475.2778
9150  34878.0556  58135.8333
9151  34917.2222  58386.9444


In [11]:
posiciones=[(float(x),float(y)) for x, y in zip(df['x'],df['y'])]
pos = np.asarray(posiciones)
max_pos = pos.max(0)
min_pos = pos.min(0)
ratio_pos = max_pos - min_pos
pos = (pos-min_pos)/ratio_pos
posiciones = map(tuple, pos)
posiciones=posiciones[:1000]
print (posiciones)

[(0.43563856795998135, 0.46515679523546483), (0.38515901122144475, 0.26045296038406435), (0.90156486632855637, 0.97473867765633926), (0.48207975822101995, 0.73344947688754258), (0.41023893850556392, 0.34098432289370101), (0.38465421465456412, 0.77874564595600293), (0.16658253390551989, 0.058362371181816065), (0.38313982798269575, 0.49738676219056938), (0.057041899992890652, 0.57665505152727359), (0.31802120109235615, 0.31445993260358884), (0.31802120109235615, 0.31445993260358884), (0.10701666016454663, 0.62108013871268308), (0.28773346765498892, 0.55052264730056211), (0.48561332813163732, 0.48519163673378329), (0.46340232159171901, 0.34059233683030032), (0.41544674465881198, 0.37891986128731686), (0.36042402790467032, 0.36324041875128998), (0.41102978454212041, 0.37846980379524348), (0.29581019758121135, 0.77090592468798946), (0.21049974738970242, 0.27090592207474895), (0.13326602712441593, 0.56620208983658893), (0.38644623989253285, 0.25965447431995553), (0.3715295311746295, 0.373693

Ahora ya hemos puesto el formato de las posiciones y normalizado los datos. A continuación declararemos la matriz de costes.

#Cálculo de matriz de costes

In [13]:
from scipy.spatial.distance import euclidean as distance

def get_matrix_costes(posiciones):
    num = len(posiciones)
    costes = np.array([distance(posiciones[c1],posiciones[c2]) for c1 in range(num) for c2 in range(num)])
    return costes.reshape((num, num))

Hemos declarado la función de la matriz de costes y ahora llamaremos con los datos de las posiciones para calcular la matriz de costes asociada.

In [14]:
costes = get_matrix_costes(posiciones)
print (costes)

[[ 0.          0.21083606  0.69047883 ...,  0.32414658  0.2792496
   0.29056059]
 [ 0.21083606  0.          0.88140745 ...,  0.37827615  0.4061588
   0.39974159]
 [ 0.69047883  0.88140745  0.         ...,  0.90236424  0.77748313
   0.81120306]
 ..., 
 [ 0.32414658  0.37827615  0.90236424 ...,  0.          0.13229269
   0.09829191]
 [ 0.2792496   0.4061588   0.77748313 ...,  0.13229269  0.          0.03449672]
 [ 0.29056059  0.39974159  0.81120306 ...,  0.09829191  0.03449672  0.        ]]


Esta es la matriz de costes obtenida una vez que se han normalizado los datos.

#FITNESS

In [15]:
def fitness(solucion, costes):
    sum = 0
    
    for i in range(num_ciudades-1):
        c1 = solucion[i]
        c2 = solucion[i+1]
        sum += costes[c1,c2]
    # Volver a la ciudad inicial
    ciudad_final = solucion[num_ciudades-1]
    ciudad_origen = solucion[0]
    sum += costes[ciudad_final, ciudad_origen]
    return sum

## Búsqueda aleatoria

In [16]:
def random_sol(num):
    return np.random.permutation(num)

In [17]:
from sys import maxint

def random_search(costes, evaluations):
    num = costes.shape[0]
    best = maxint
    best_sol = []
    
    for r in range(evaluations):
        solucion = random_sol(num)
        fitness_sol = fitness(solucion, costes)
        if(fitness_sol < best):
                best = fitness_sol
                best_sol = solucion
            
    return best_sol

In [18]:
best_solucion =  random_search(costes, 50000)

#Algoritmo de GRASP

In [19]:
#Sea pendientes un vector de ciudades por v i s i t a r costes la matriz de costes y c1 la ciudad destino con menor distancia c2 se calcula como:
#distancias = costes[c1] 
#distancias_destino = distancias[pendientes] 
#c2= pendientes[distancias_destino.argmin()]

def construccion_solucion_greedy(costes):
     
    pendientes= np.arange(1000)
    # print "Pendientes:"
    #print pendientes
    pendientes=np.delete(pendientes,0)
    #print "Eliminamos el primero:"
    #print pendientes
    camino_solucion= []
    c1=0
    camino_solucion.append(c1)
    num_ciudades = len(pendientes)
    
    for i in range(num_ciudades):

    
        distancias=costes[c1]
        distancias_destino = distancias[pendientes]
        distancias_destino2 = distancias_destino[:]
        c2 = pendientes[distancias_destino.argmin()]
        idCiudadesCercanas= []
        
        if (len(distancias_destino2) >= 1):
            c1=0
            id1= distancias_destino2.argmin()
            idCiudadesCercanas.append(id1)
            distancias_destino2=np.delete(distancias_destino2,id1)
        
        
        if (len(distancias_destino2) >= 2):
            c1=idCiudadesCercanas[randint(0,1)];
            id2= distancias_destino2.argmin();
            idCiudadesCercanas.append(id2)
            distancias_destino2=np.delete(distancias_destino2,id2)
      
            
        if (len(distancias_destino2) >= 3):
            c1=idCiudadesCercanas[randint(0,2)];
            id3= distancias_destino2.argmin();
            idCiudadesCercanas.append(id3)
            distancias_destino2=np.delete(distancias_destino2,id3)
       
        
        #Devolver una de las 3 ciudades obtenidas de forma aleatoria.
        #c1=idCiudadesCercanas[randint(0,3)];
        camino_solucion.append(c1)
        pendientes=np.delete(pendientes, c1)
        
    # Conjunto de vertices que son la solución
    return camino_solucion

In [20]:
A=construccion_solucion_greedy(costes)
print A

[0, 750, 749, 748, 747, 822, 817, 812, 447, 492, 916, 682, 678, 718, 524, 52, 51, 50, 49, 713, 610, 625, 616, 971, 573, 564, 555, 547, 539, 531, 523, 516, 509, 499, 492, 486, 480, 474, 397, 392, 387, 110, 104, 91, 82, 77, 169, 159, 4, 534, 507, 484, 296, 283, 270, 426, 407, 104, 95, 87, 81, 75, 69, 791, 148, 131, 493, 373, 349, 325, 301, 278, 309, 113, 98, 361, 331, 310, 492, 638, 573, 511, 11, 508, 815, 737, 826, 96, 823, 741, 894, 809, 730, 191, 166, 143, 595, 325, 339, 478, 288, 259, 231, 211, 184, 155, 499, 30, 27, 24, 21, 18, 164, 349, 397, 324, 280, 556, 474, 478, 255, 218, 182, 247, 209, 442, 598, 487, 416, 351, 297, 294, 250, 259, 357, 299, 252, 213, 324, 90, 118, 95, 49, 41, 33, 452, 376, 305, 249, 211, 193, 154, 121, 20, 699, 566, 447, 357, 186, 146, 113, 621, 491, 675, 538, 65, 49, 579, 450, 353, 276, 218, 478, 376, 700, 105, 79, 4, 3, 2, 76, 55, 37, 24, 572, 531, 181, 202, 146, 593, 342, 323, 239, 489, 112, 466, 348, 258, 712, 616, 461, 769, 587, 433, 306, 715, 541, 38, 109

In [21]:
from sys import maxint

def grasp(costes, evaluations):
    num = costes.shape[0]
    best = maxint
    best_sol = []
    
    for r in range(evaluations):
        solucion = construccion_solucion_greedy(costes) 
        fitness_solucion=fitness(solucion, costes)
        if (fitness_solucion < best):
            best = fitness_solucion
            best_sol = solucion
            
    return best_sol

In [22]:
distancia1=0
distancia2=0
    
for r in range(10):
    distancia1+= fitness(grasp(costes,500),costes)
    distancia2+= fitness(random_search(costes,500),costes)
    
distanciat1= distancia1/10
distanciat2= distancia2/10

print "Distancia del algoritmo grasp es: "+"%.2f" % distanciat1
print "La distancia del algoritmo aleatorio es: "+"%.2f" % distanciat2

Distancia del algoritmo grasp es: 281.29
La distancia del algoritmo aleatorio es: 285.67
